In [141]:
import tensorflow as tf
from tensorflow import keras as ke

vocab_size = 23940
dim = 200

def word2vec(vocab_size = vocab_size,h_size = dim):
    model = ke.Sequential()
    model.add(ke.layers.Dense(h_size, input_shape = (vocab_size,)))
    model.add(ke.layers.Dense(vocab_size,activation = ke.activations.softmax))
    return model

def make_reduced_model(in_out_tuple,model):
    in_out_tuple[0].sort() #sorting for later naming reasons
    in_out_tuple[1].sort()
    print(in_out_tuple)
    in_layer,out_layer = model.layers
    in_params = in_layer.get_weights()
    out_params = out_layer.get_weights()
    
    in_lst,out_lst = in_out_tuple
    
    in_weights = in_params[0][in_lst] #shape (2,200)
    in_bias = in_params[1] #shape (200,)
    out_weights = out_params[0][:,out_lst] #shape (200,1)
    out_bias = out_params[1][out_lst] #shape (1,)
    
    reduced_model = ke.Sequential()
    reduced_model._name = str(in_out_tuple)
    #making layers of the shapes of the extracted parameters
    h_layer = ke.layers.Dense(in_weights.shape[1], input_shape=(in_weights.shape[0],))
    out_layer = ke.layers.Dense(out_weights.shape[1], ke.activations.softmax)
    reduced_model.add(h_layer)
    reduced_model.add(out_layer)

    
    #setting weights
    reduced_model.layers[0].set_weights([in_weights,in_bias])
    reduced_model.layers[1].set_weights([out_weights,out_bias])
    reduced_model.compile(loss="categorical_crossentropy",optimizer='adam', metrics=['accuracy'])
    return reduced_model

def update_word2vec(reduced_model, model):
    in_out_tuple = eval(reduced_model.name)
    
    in_layer,out_layer = model.layers
    in_params = in_layer.get_weights()
    out_params = out_layer.get_weights()
    
    in_lst,out_lst = in_out_tuple
    
    in_weights, in_bias = in_params
    out_weights, out_bias = out_params
    
    reduced_in_layer, reduced_out_layer = reduced_model.layers
    reduced_in_params = reduced_in_layer.get_weights()
    reduced_out_params = reduced_out_layer.get_weights()
    
    reduced_in_weights, reduced_in_bias = reduced_in_params
    reduced_out_weights, reduced_out_bias = reduced_out_params
    
    #changing weights
    in_weights[in_lst] = reduced_in_weights #replace rows
    in_bias = reduced_in_bias #replace all of it
    out_weights[:,out_lst] = reduced_out_weights #replace columns
    out_bias[out_lst] = reduced_out_bias #replace parts of 1d array
    
    #update word2vec
    in_layer.set_weights([in_weights,in_bias])
    out_layer.set_weights([out_weights,out_bias])
    assert (model.layers[0].get_weights()[0] == in_layer.get_weights()[0]).all()
    
    return model

word2vec_model = word2vec()
word2vec_model.compile(loss="categorical_crossentropy",optimizer='adam', metrics=['accuracy'])

In [ ]:
def train_on_tuples_batch(tuples_batch,model):
    independent_tups = []
    for tup in tuples_batch:
        indpendent_tups
        
        

In [142]:
red_model = make_reduced_model(([2,1],[1]),word2vec_model)
w,b = red_model.layers[0].get_weights()
w_c = w.copy()
w[1,1] += 0.5
print((w_c == w).all())
red_model.layers[0].set_weights([w,b])

([1, 2], [1])
False


In [143]:
update_word2vec(red_model,word2vec_model)

In [144]:
word2vec_model